<a href="https://colab.research.google.com/github/MBolcao/BigData/blob/main/TrabBigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grafico de usuarios por semana

In [ ]:
import pandas as pd
import plotly.express as px

tabela = pd.read_csv('ArqLog2.csv',  parse_dates=['LogonData', 'LogoffData'], dayfirst=False)
tabela['DiaDaSemana'] = tabela['LogonData'].dt.day_name()
# Remover usuários duplicados
df_unicos = tabela.drop_duplicates(subset=['DiaDaSemana', 'Usuario'])
print(df_unicos)

# Agrupar por dia da semana
agrupado = df_unicos.groupby('DiaDaSemana').size().reset_index(name='Contagem')

# Ordenar os dias da semana
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
agrupado['DiaDaSemana'] = pd.Categorical(agrupado['DiaDaSemana'], categories=dias_ordenados, ordered=True)
agrupado = agrupado.sort_values('DiaDaSemana')

print(agrupado)


# Criar o gráfico de barras
fig = px.bar(agrupado, x='DiaDaSemana', y='Contagem', title='Contagem de Usuários Únicos por Dia da Semana',
             labels={'Contagem': 'Número de Usuários Únicos', 'DiaDaSemana': 'Dia da Semana'},
             color='Contagem', color_continuous_scale='Blues')

fig.show()


      LogonData LogonHora LogoffData LogoffHora   Usuario DiaDaSemana
0    2024-06-21  07:20:24 2024-06-21   15:23:30  E012U009      Friday
1    2024-06-21  14:35:51 2024-06-21   15:22:10  E012U006      Friday
2    2024-06-21  15:15:28 2024-06-21   15:17:35  E016U001      Friday
4    2024-06-21  16:04:47 2024-06-21   18:12:41  E012U002      Friday
5    2024-06-21  16:04:48 2024-06-21   16:53:37  E012U004      Friday
...         ...       ...        ...        ...       ...         ...
1559 2024-07-13  12:04:55 2024-07-13   12:45:28  E029U001    Saturday
1590 2024-07-15  09:05:44 2024-07-15   14:20:00  E016U003      Monday
1597 2024-07-15  10:47:40 2024-07-15   10:48:24  E030U001      Monday
1598 2024-07-15  10:48:52 2024-07-15   10:49:15  E030U002      Monday
1922 2024-07-19  08:12:32 2024-07-19   12:32:41  E029U002      Friday

[139 rows x 6 columns]
  DiaDaSemana  Contagem
1      Monday        25
5     Tuesday        23
6   Wednesday        22
4    Thursday        23
0      Friday   

Grafico de usuarios por dia


In [ ]:
import pandas as pd
import plotly.express as px

tabela = pd.read_csv('ArqLog2.csv',  parse_dates=['LogonData', 'LogoffData'], dayfirst=False)
tabela['DiaDaSemana'] = tabela['LogonData'].dt.day_name()
# Remover usuários duplicados
df_unicos = tabela.drop_duplicates(subset=['LogonData', 'Usuario'])
print(df_unicos)

# Agrupar por dia da semana
agrupado = df_unicos.groupby('LogonData').size().reset_index(name='Contagem')

# Criar o gráfico de barras
fig = px.bar(agrupado, x='LogonData', y='Contagem', title='Contagem de Usuários Únicos por Dia',
             labels={'Contagem': 'Número de Usuários Únicos', 'Data': 'Data'},
             color='Contagem', color_continuous_scale='Blues')

fig.update_xaxes(type='category')
fig.show()

      LogonData LogonHora LogoffData LogoffHora   Usuario DiaDaSemana
0    2024-06-21  07:20:24 2024-06-21   15:23:30  E012U009      Friday
1    2024-06-21  14:35:51 2024-06-21   15:22:10  E012U006      Friday
2    2024-06-21  15:15:28 2024-06-21   15:17:35  E016U001      Friday
4    2024-06-21  16:04:47 2024-06-21   18:12:41  E012U002      Friday
5    2024-06-21  16:04:48 2024-06-21   16:53:37  E012U004      Friday
...         ...       ...        ...        ...       ...         ...
2060 2024-07-23  06:39:17 2024-07-23   06:40:00  E011U001     Tuesday
2062 2024-07-23  06:41:51 2024-07-23   06:42:19  E012U006     Tuesday
2066 2024-07-23  07:11:55 2024-07-23   08:41:04  E020U001     Tuesday
2070 2024-07-23  08:35:09 2024-07-23   09:46:39  E019U003     Tuesday
2072 2024-07-23  08:56:30 2024-07-23   10:31:45  E012U001     Tuesday

[474 rows x 6 columns]


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Grafico de usuario por data e por hora logados no sistema

In [ ]:
import pandas as pd
import plotly.express as px

# Definir datas de início e fim para filtro
data_inicio = pd.to_datetime('2024-06-21')  # Substitua pela data de início desejada
data_fim = pd.to_datetime('2024-06-24')     # Substitua pela data de fim desejada


df = pd.read_csv('ArqLog2.csv')

# Converter as colunas de data e hora para o formato datetime
df['Logon'] = pd.to_datetime(df['LogonData'] + ' ' + df['LogonHora'])
df['Logoff'] = pd.to_datetime(df['LogoffData'] + ' ' + df['LogoffHora'])


# Filtrar os dados entre a data de início e fim
df = df[(df['Logon'].dt.date >= data_inicio.date()) & (df['Logoff'].dt.date <= data_fim.date())]

# Criar uma coluna para cada hora entre logon e logoff
df_expanded = df.copy()
df_expanded['Hora'] = df_expanded.apply(lambda row: pd.date_range(start=row['Logon'], end=row['Logoff'], freq='H'), axis=1)
df_expanded = df_expanded.explode('Hora')

# Criar colunas para data e hora separadamente
df_expanded['Data'] = df_expanded['Hora'].dt.date
df_expanded['Hora'] = df_expanded['Hora'].dt.hour

# Remover usuários duplicados
df_unicos = df_expanded.drop_duplicates(subset=['Data', 'Hora', 'Usuario'])
# Agrupar por data e hora
#agrupado = df_unicos.groupby(['Data', 'Hora']).size().reset_index(name='Contagem')
agrupado = df_unicos.groupby(['Data', 'Hora']).nunique()['Usuario'].reset_index(name='Contagem')

# Criar o gráfico de calor
#fig = px.density_heatmap(agrupado, x='Hora', y='Data', z='Contagem', title='Usuários Únicos por Dia e Hora',
#                         labels={'Contagem': 'Número de Usuários Únicos'},
#                         color_continuous_scale='Viridis')

fig = px.bar(agrupado, x='Hora', y='Contagem', color='Data', barmode='group',
             title='Usuários Únicos por Hora e Dia',
             labels={'Contagem': 'Número de Usuários Únicos', 'Hora': 'Hora do Dia'})


fig.show()

#print(agrupado)
